In [1]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

import json
import gzip

from os import walk
from scipy.spatial import KDTree

# !pip install geopy
# !pip install phonenumbers
# !pip install pycountry

import geopy.distance
import phonenumbers
import pycountry



In [2]:
pd.options.display.max_columns = 100

In [3]:
path = r"../src/data/Hotel"
file_path_min3 = path + r"/Hotel_minimum3/geo_preprocessed"
file_path_top100 = path + r"/Hotel_top100/geo_preprocessed"

files_min3 = os.listdir(file_path_min3)
files_top100 = os.listdir(file_path_top100)

In [4]:
print(len(files_min3))
print(len(files_top100))

1043
53


In [5]:
LB_min3 = []

for lb in files_min3:
    with gzip.open(file_path_min3 + '/' + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LB_min3.append(lineData)
df_min3 = pd.DataFrame(LB_min3)

In [6]:
len(df_min3)

207931

In [7]:
LB_top100 = []

for lb in files_top100:
    with gzip.open(file_path_top100 + '/' + lb, 'r') as dataFile:
        for line in dataFile:
            lineData = json.loads(line.decode('utf-8'))
            LB_top100.append(lineData)
df_top100 = pd.DataFrame(LB_top100)

In [8]:
len(df_top100)

316809

## Concatenate Dataframes

In [9]:
df_all = pd.concat([df_min3, df_top100], axis = 0, ignore_index = True)

In [10]:
len(df_all)

524740

In [11]:
df_all.tail()

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,...,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee
524735,5185,Supalai Resort & Spa Phuket,"Moo 6 Ao Po Lang Daeng Road Paklok Thala,",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/thailand/p...,NaN,NaN,NaN,NaN,NaN,+66 763 02302,NaN,"{'reviewcount': '140', 'ratingvalue': '3.5E0'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524736,5186,Oba Star Hotel,"Oba Mahallesi, Obagöl Mevkii, Mesut Cd., 07640...",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ant...,NaN,NaN,NaN,NaN,NaN,0090 242 514 32 83,NaN,"{'ratingvalue': '4.5E0', 'reviewcount': '1251'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524737,5187,Hotel Askoc,"Istasyon Arka Sokak No 15Sirkeci 34420, Istanbul",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ist...,NaN,NaN,NaN,NaN,NaN,902125118089,NaN,"{'reviewcount': '148', 'ratingvalue': '3.5E0'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524738,5188,Hotel Xapala,"Carrer d'Amílcar, 5, 07600 El Arenal, Illes Ba...",NaN,NaN,{'ratingvalue': '0'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/spain/bale...,NaN,NaN,NaN,NaN,NaN,None,NaN,"{'ratingvalue': '4.0E0', 'reviewcount': '82'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524739,5189,Hotel Best Western Amber,"Muhsine Hatun Mah.Ciftegelinler Cad.No:3/7, Is...",NaN,NaN,{'ratingvalue': '4'},NaN,NaN,NaN,NaN,https://www.onthebeach.co.uk/hotels/turkey/ist...,NaN,NaN,NaN,NaN,NaN,+90 212 518 48 01,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Keep where dataframe has non-zero telephone numbers AND non-zero country codes

In [12]:
df_clean = df_all[df_all["addresscountry"].notna()]
df_clean = df_clean[df_clean["telephone"].notna()]
len(df_clean)

158543

## Format longitudes AND latitudes

In [13]:
lon = "longitude"
lat = "latitude"

In [14]:
# Remove entries that are not numbers or cannot be convertred to one number (list etc.)
longitudes = df_clean[lon].to_numpy()
latitudes = df_clean[lat].to_numpy()
deleteList = []
i = 0

for value in longitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1

i = 0
for value in latitudes:
    if ((isinstance(value, str) == False) & (isinstance(value, float) == False)):
        deleteList.append(i)
    i = i + 1    

df_clean.drop(df_clean.index[deleteList], axis = 0, inplace = True)        

### Format longitude and latitude

In [15]:
longArray = df_clean[lon].to_numpy().astype(str)
longArray = np.char.replace(longArray, ',', '.')
longArray = np.char.replace(longArray, '--', '-')
df_clean[lon] = longArray
df_clean[lon] = pd.to_numeric(df_clean[lon], errors='coerce')

latArray = df_clean[lat].to_numpy().astype(str)
latArray = np.char.replace(latArray, ',', '.')
latArray = np.char.replace(latArray, '--', '-')
df_clean[lat] = latArray
df_clean[lat] = pd.to_numeric(df_clean[lat], errors='coerce')

In [16]:
# Remove the entries that were set to NaN because of other errors
df_clean = df_clean[df_clean["longitude"].notna()]
df_clean = df_clean[df_clean["latitude"].notna()]
# Make sure to only include valid longitudes and latitudes
df_clean = df_clean.loc[(df_clean[lat] >= -90) & (df_clean[lat] <= 90)]
df_clean = df_clean.loc[(df_clean[lon] >= -180) & (df_clean[lon] <= 80)]

In [17]:
len(df_clean)

67570

## Second preprocessing step

### Remove non-digits from telephone numbers

In [18]:
def remove_non_digits(string):
    cleaned = re.sub('[^0-9]','', string)
    return cleaned

In [19]:
df_clean['telephone_'] = df_clean['telephone'].astype('str').apply(remove_non_digits)

### Extract country codes to ISO-2 format using ``pycountry``

In [20]:
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

countries


{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [21]:
# fuction to modify the country dictionary in uppercase
def modify_dic(d):
    for key in list(d.keys()):
        new_key = key.upper()
        d[new_key] = d[key]
        print(new_key)
    return d

In [22]:
countries_upper = modify_dic(countries)

ARUBA
AFGHANISTAN
ANGOLA
ANGUILLA
ÅLAND ISLANDS
ALBANIA
ANDORRA
UNITED ARAB EMIRATES
ARGENTINA
ARMENIA
AMERICAN SAMOA
ANTARCTICA
FRENCH SOUTHERN TERRITORIES
ANTIGUA AND BARBUDA
AUSTRALIA
AUSTRIA
AZERBAIJAN
BURUNDI
BELGIUM
BENIN
BONAIRE, SINT EUSTATIUS AND SABA
BURKINA FASO
BANGLADESH
BULGARIA
BAHRAIN
BAHAMAS
BOSNIA AND HERZEGOVINA
SAINT BARTHÉLEMY
BELARUS
BELIZE
BERMUDA
BOLIVIA, PLURINATIONAL STATE OF
BRAZIL
BARBADOS
BRUNEI DARUSSALAM
BHUTAN
BOUVET ISLAND
BOTSWANA
CENTRAL AFRICAN REPUBLIC
CANADA
COCOS (KEELING) ISLANDS
SWITZERLAND
CHILE
CHINA
CÔTE D'IVOIRE
CAMEROON
CONGO, THE DEMOCRATIC REPUBLIC OF THE
CONGO
COOK ISLANDS
COLOMBIA
COMOROS
CABO VERDE
COSTA RICA
CUBA
CURAÇAO
CHRISTMAS ISLAND
CAYMAN ISLANDS
CYPRUS
CZECHIA
GERMANY
DJIBOUTI
DOMINICA
DENMARK
DOMINICAN REPUBLIC
ALGERIA
ECUADOR
EGYPT
ERITREA
WESTERN SAHARA
SPAIN
ESTONIA
ETHIOPIA
FINLAND
FIJI
FALKLAND ISLANDS (MALVINAS)
FRANCE
FAROE ISLANDS
MICRONESIA, FEDERATED STATES OF
GABON
UNITED KINGDOM
GEORGIA
GUERNSEY
GHANA
GIBRALTAR
GUI

In [23]:
countries_upper

{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [24]:
#uppercase the df_column 
df_clean["addresscountry"] = df_clean["addresscountry"].str.upper()

In [25]:
# Replace known countrires with ISO-2 format country code
for key, value in countries_upper.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

<ipython-input-25-d03c22551531>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)


In [26]:
df_clean["addresscountry"].unique()

array(['US', 'SCOTLAND', '约旦', 'AE', 'LIBAN', 'BH',
       'MUUNGANO WA FALME ZA KIARABU', 'TÜRKEI', 'BAHREIN',
       'EMIRATS ARABES UNIS', 'JO', 'LB', 'JOIEN', 'QA',
       'VEREINIGTE ARABISCHE EMIRATE', 'ИОРДАНИЯ', 'OM', 'SA', 'TURQUÍA',
       'EMIRATOS ÁRABES UNIDOS', 'الأردن', 'RD DU CG', 'لبنان', 'BAHRÉIN',
       '土耳其', 'КАТАР', '巴林', 'БАХРЕЙН', 'الإمارات العربية المتحدة',
       'العراق', 'DR OF CG', '坦桑尼亚', 'TÜRKIYE', 'IN', 'KE', 'ZM', 'FR',
       'CROATIE', 'ESPAGNE', 'ES', 'BE', 'DE', 'FI', 'AT', 'GR', 'CH',
       'MX', 'UK', 'IT', 'MT', 'USA', 'HU', 'SE', 'SI', 'PT',
       'CZECH REPUBLIC', 'NL', 'SK', 'CA', 'EE', 'BR', 'IE', 'IL',
       'ÖSTERREICH', 'TR', 'LV', 'AW', 'ZA', 'LY', 'GB', 'CO', 'AM', 'PE',
       'DK', 'NO', 'CÔTE D&#039;IVOIRE', 'CR', '荷兰', 'NOORWEGEN',
       'HRVATSKA', 'RU', 'VEREINIGTES KÖNIGREICH', 'ROMIA', 'PA', 'GA',
       'LT', 'CL', 'HR', 'ML', 'REGNO UNITO', 'UZ', '比利时', 'TN', 'IS',
       'MU', 'PL', 'GE', 'RS', 'RW', 'DZ', 'NEIA', 'KZ', '

## Manually normalize countries which do not exist in country package

In [27]:
df_clean["addresscountry"].value_counts().head(30)

US         20229
USA        10705
IN          8202
IT          2429
DE          2247
FR          1939
GB          1789
CA          1539
DEU         1376
RO          1054
MX           990
EE. UU.      977
ES           772
UK           717
IRAN         634
ITA          526
JM           495
DK           484
AT           392
NO           360
AE           356
GR           335
BR           253
PT           231
TR           222
PL           221
SPANIEN      216
HU           192
TÜRKEI       171
HR           162
Name: addresscountry, dtype: int64

In [28]:
country_dictionary = {
                      "UNITED STATES": "US",
                      "USA":"US",
                      "UNITED KINGDOM": "GB",
                      "UK": "GB",
                      "CANADA": "CA",
                      "AUSTRALIA": "AU",
                      "UNITED ARAB EMIRATES":"AE",
                      "INDIA" : "IN",
                      "NEW ZEALAND": "NZ",
                      "SVERIGE" : "SE",
                      "DEUTSCHLAND": "DE",
                      "RUSSIA": "RU",
                      "ITALIA": "IT",
                      "IRAN": "IR",
                      ", IN" : "IN",
                      "ENGLAND": "GB"
                    }
for key, value in country_dictionary.items():
    df_clean["addresscountry"] = df_clean["addresscountry"].str.replace(key, value)

## In this manual step we save about 43.000 extra datapoints

## Remove non-covered countries

### There are still some uncovered cases left which have to be removed

In [29]:
df_clean.reset_index(inplace=True)

In [30]:
liste = []
for i, row in enumerate(df_clean["addresscountry"]):
    if len(row) > 2:
        liste.append(i)
        
df_clean = df_clean.drop(liste)        

In [31]:
df_clean["addresscountry"].unique()

array(['US', '约旦', 'AE', 'BH', 'JO', 'LB', 'QA', 'OM', 'SA', '巴林', 'IN',
       'KE', 'ZM', 'FR', 'ES', 'BE', 'DE', 'FI', 'AT', 'GR', 'CH', 'MX',
       'GB', 'IT', 'MT', 'HU', 'SE', 'SI', 'PT', 'NL', 'SK', 'CA', 'EE',
       'BR', 'IE', 'IL', 'TR', 'LV', 'AW', 'ZA', 'LY', 'CO', 'AM', 'PE',
       'DK', 'NO', 'CR', '荷兰', 'RU', 'PA', 'GA', 'LT', 'CL', 'HR', 'ML',
       'UZ', 'TN', 'IS', 'MU', 'PL', 'GE', 'RS', 'RW', 'DZ', 'KZ', 'BZ',
       'ET', 'KW', 'UY', '挪威', 'GD', 'SN', 'UA', 'GT', 'AZ', 'SL', '芬兰',
       'PK', 'HN', 'PR', 'BG', 'LK', 'SC', 'CY', 'TH', 'MZ', 'MV', 'ID',
       'CN', 'MY', 'KH', 'MA', 'RO', 'LU', 'CZ', 'TC', 'AR', 'JM', 'BL',
       'AI', 'BS', 'ZW', 'TZ', 'ME', 'VC', 'NA', 'BB', 'NI', 'KY', 'LC',
       'EG', 'IR', 'KG', 'MC', 'KN', 'EC', 'SV', 'DM', 'CK', 'PF', 'DJ',
       'GP', 'BY', 'AD', 'UG', 'BM', 'GH', 'BW', 'P', 'AL', 'MK', 'DO',
       '美国', '德国', 'LS', 'CE', 'TT', 'TJ', 'WS', 'BA', 'HT', 'MW', 'PY',
       '中国', 'PH', 'AU', 'HK', 'SG', 'GN', 'SR', 'FJ

## Drop empty phonenumbers and too lenghty phone numbers

In [32]:
df_clean = df_clean[df_clean["telephone_"] != "" ]

In [33]:
liste = []
df_clean.reset_index(inplace=True)
for row_index in df_clean.index:
    if len(df_clean.iloc[row_index]["telephone_"])>18:
        liste.append(row_index)

In [34]:
df_clean.drop(labels = liste, inplace = True)

In [35]:
df_clean = df_clean.drop(columns = ["level_0","index"])
df_clean.tail()

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,...,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_
55304,9996,None,"{'streetaddress': 'Centre Island, Waterways, L...",NaN,NaN,NaN,NaN,NaN,53.290300,-2.894400,https://www.ihg.com/hotelindigo/destinations/c...,CH65 2AL,"Centre Island, Waterways, Lower Mersey Street",None,Ellesmere Port,英国,44-151-3568111,"{'longitude': '-2.8944E0', 'latitude': '5.3290...","{'bestrating': '5.0', 'ratingvalue': '4.6', 'r...",None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{'name': 'InterContinental Hotels Group'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441513568111
55305,9999,None,"{'streetaddress': '12217 4th Street', 'address...",NaN,NaN,NaN,NaN,NaN,55.737782,-120.216293,https://www.ihg.com/holidayinnexpress/destinat...,V1G 0A4,12217 4th Street,COLOMBIE BRITANNIQUE,Dawson Creek,CA,1-250-7827700,"{'longitude': '-1.20216293E2', 'latitude': '5....","{'bestrating': '5.0', 'ratingvalue': '4.8', 'r...",None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{'name': 'InterContinental Hotels Group'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12507827700
55306,10002,None,"{'postalcode': '79938', 'addresslocality': 'El...",NaN,NaN,NaN,NaN,NaN,31.797262,-106.266840,https://www.ihg.com/destinations/tw/zh/united-...,79938,3500 Joe Battle Blvd,得克萨斯州,El Paso,美国,1-915-8556161,"{'latitude': '3.17972624E1', 'longitude': '-1....","{'ratingcount': '264', 'bestrating': '5.0', 'r...",None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{'name': 'InterContinental Hotels Group'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19158556161
55307,10003,None,"{'streetaddress': '1091 Doniphan Park Circle',...",NaN,NaN,NaN,NaN,NaN,31.815977,-106.556546,https://www.ihg.com/destinations/tw/zh/united-...,79922,1091 Doniphan Park Circle,得克萨斯州,El Paso,美国,1-915-5812447,"{'latitude': '3.18159774E1', 'longitude': '-1....","{'bestrating': '5.0', 'ratingvalue': '4.6', 'r...",None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,{'name': 'InterContinental Hotels Group'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [36]:
len(df_clean)

54758

## Define normalizer for telephone package phonenumbers

In [37]:
def normalizer(entity):
    number = entity["telephone_"]
    address_country = entity["addresscountry"]
    phone_number = phonenumbers.parse(number, address_country)
    return phone_number

## Finally normalizing phone numbers in E.164 format

### Ignore those which can not be identified and replace as ``unknown``

In [38]:
df_clean.reset_index(inplace=True)
phone_objects =[]
#index = []
for row_index in df_clean.index:
    try:
        phone_object =  normalizer(df_clean.iloc[row_index])
        #index.append(row_index)
        phone_objects.append(phone_object)
    except:
        phone_objects.append("unknown")
    
    

In [39]:
len(phone_objects)

54758

In [40]:
df_clean["phone_object"] = pd.Series(phone_objects)

In [41]:
df_clean = df_clean.drop(columns = "index")
df_clean.head()

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,...,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object
0,0,布利华德阿尔杨罗塔纳酒店,"{'addresslocality': '安曼', 'streetaddress': 'Th...",NaN,JOD 69 - JOD 1430,{'ratingvalue': '5'},NaN,NaN,31.964408,35.906534,https://cn-mobile.rotana.com/arjaanhotelapartm...,NaN,"The Boulevard, Rafiq Al Hariri Avenue, Abdali ...",NaN,安曼,约旦,+962 6 520 4444,"{'longitude': '3.5906534E1', 'latitude': '3.19...","{'ratingvalue': '4.44', 'reviewcount': '3779'}",NaN,+962 6 520 4555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96265204444,unknown
1,1,Al Bandar Arjaan - Creek,"{'addresscountry': 'United Arab Emirates', 'st...",NaN,AED 224 - AED 2622,{'ratingvalue': '4'},NaN,NaN,25.259316,55.319038,https://mobile.rotana.com/arjaanhotelapartment...,NaN,Baniyas Street,NaN,Dubai,AE,+971 4 704 2299,"{'latitude': '2.5259316E1', 'longitude': '5.53...","{'reviewcount': '1225', 'ratingvalue': '4.68'}",NaN,+971 4 704 2233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97147042299,Country Code: 971 National Number: 47042299
2,4,Hala Arjaan by Rotana,"{'addresscountry': 'United Arab Emirates', 'ad...",NaN,AED 168 - AED 395,{'ratingvalue': '4'},NaN,NaN,24.501032,54.377946,https://www.rotana.com/arjaanhotelapartments/u...,NaN,"Hamdan Street, Al Zahiyah Area, PO Box 53066,",NaN,Abu Dhabi,AE,+971 2 418 0000,"{'longitude': '5.437794551253319E1', 'latitude...","{'reviewcount': '1873', 'ratingvalue': '4.48'}",NaN,+971 2 418 0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97124180000,Country Code: 971 National Number: 24180000
3,6,ART Rotana,"{'addresscountry': 'Bahrain', 'streetaddress':...",NaN,BHD 28 - BHD 1053,{'ratingvalue': '5'},NaN,NaN,26.289799,50.659563,https://www.rotana.com/rotanahotelandresorts/b...,NaN,"Al Fanar Avenue Road 5722, Building 2780",NaN,Manama,BH,+973 16000111,"{'latitude': '2.6289799107E1', 'longitude': '5...","{'ratingvalue': '4.54', 'reviewcount': '6520'}",NaN,+973 16000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [42]:
unknown_rows = df_clean[df_clean["phone_object"] == "unknown"].index

In [43]:
df_clean = df_clean.drop(unknown_rows)

In [44]:
len(df_clean)

54488

## Check whether phonenumbers are valid

In [45]:
df_valid_numbers = df_clean[df_clean["phone_object"].apply(phonenumbers.is_valid_number)]

In [46]:
len(df_valid_numbers)

49383

## Next step: Format every telephone number into unique E.164 format

In [47]:
#phonenumbers.format_number(df_valid_numbers["phone_object"][0], phonenumbers.PhoneNumberFormat.E164)

In [48]:
df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda objects: phonenumbers.format_number(objects, phonenumbers.PhoneNumberFormat.E164))

<ipython-input-48-28dce20706cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers["E.164 format"] = df_valid_numbers["phone_object"].apply(lambda objects: phonenumbers.format_number(objects, phonenumbers.PhoneNumberFormat.E164))


In [49]:
len(df_valid_numbers)

49383

In [50]:
df_valid_numbers.head()

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,...,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object,E.164 format
1,1,Al Bandar Arjaan - Creek,"{'addresscountry': 'United Arab Emirates', 'st...",NaN,AED 224 - AED 2622,{'ratingvalue': '4'},NaN,NaN,25.259316,55.319038,https://mobile.rotana.com/arjaanhotelapartment...,NaN,Baniyas Street,NaN,Dubai,AE,+971 4 704 2299,"{'latitude': '2.5259316E1', 'longitude': '5.53...","{'reviewcount': '1225', 'ratingvalue': '4.68'}",NaN,+971 4 704 2233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97147042299,Country Code: 971 National Number: 47042299,+97147042299
2,4,Hala Arjaan by Rotana,"{'addresscountry': 'United Arab Emirates', 'ad...",NaN,AED 168 - AED 395,{'ratingvalue': '4'},NaN,NaN,24.501032,54.377946,https://www.rotana.com/arjaanhotelapartments/u...,NaN,"Hamdan Street, Al Zahiyah Area, PO Box 53066,",NaN,Abu Dhabi,AE,+971 2 418 0000,"{'longitude': '5.437794551253319E1', 'latitude...","{'reviewcount': '1873', 'ratingvalue': '4.48'}",NaN,+971 2 418 0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97124180000,Country Code: 971 National Number: 24180000,+97124180000
3,6,ART Rotana,"{'addresscountry': 'Bahrain', 'streetaddress':...",NaN,BHD 28 - BHD 1053,{'ratingvalue': '5'},NaN,NaN,26.289799,50.659563,https://www.rotana.com/rotanahotelandresorts/b...,NaN,"Al Fanar Avenue Road 5722, Building 2780",NaN,Manama,BH,+973 16000111,"{'latitude': '2.6289799107E1', 'longitude': '5...","{'ratingvalue': '4.54', 'reviewcount': '6520'}",NaN,+973 16000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97316000111,Country Code: 973 National Number: 16000111,+97316000111
4,8,Rose Rayhaan by Rotana,"{'addresscountry': 'United Arab Emirates', 'ad...",NaN,AED 125 - AED 1906,{'ratingvalue': '4'},NaN,NaN,25.211458,55.277005,https://www.rotana.com/rayhaanhotelandresorts/...,NaN,Sheikh Zayed Road,NaN,Dubai,AE,+971 4 323 0111,"{'longitude': '5.527700453996658E1', 'latitude...","{'reviewcount': '11863', 'ratingvalue': '4.54'}",NaN,+971 4 323 0222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## After formatting phone numbers into unified format we can group by phone numbers to identify clusters

In [51]:
df_valid_numbers["E.164 format"].value_counts().sort_values().tail(100)

+914067696769      13
+911146833333      13
+911140650000      13
+919351230410      13
+917774090289      13
                 ... 
+917042424242     473
+982144698615     634
+40364431800      938
+919313931393    5260
+18555161090     6955
Name: E.164 format, Length: 100, dtype: int64

## As one can see from the geo locations this is a *successful* match!

In [52]:
pd.set_option('display.max_columns', 500)
df_valid_numbers[df_valid_numbers["E.164 format"] == "+442084681087"][:5]

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,breadcrumb,datepublished,articlebody,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object,E.164 format


## Note we also have many non-matches which is why we need geo-locations

## Adding the matching telephone numbers in a new column

In [53]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, r = 0):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r)
    return idx

df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
df_valid_numbers.reset_index(drop=True, inplace=True)
df_valid_numbers['indexValue'] = df_valid_numbers.index
telephoneArray = df_valid_numbers['telephoneNorm'].to_numpy().astype('int64')
fillArray = np.full(len(telephoneArray), 1)
tupleArray = np.array((telephoneArray, fillArray)).T.astype('int64')


# create new column with all matching points
tree = createKDTree(tupleArray)
idx = queryTree(tree, tupleArray[0])

# Search for the closest neighbour in all of the points
df_valid_numbers['MatchingNumbers'] = df_valid_numbers.apply(lambda row: queryTree(tree,[row['telephoneNorm'], 1]), axis=1) 

<ipython-input-53-623d5e3ddf8f>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-53-623d5e3ddf8f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_numbers['telephoneNorm'] = df_valid_numbers['E.164 format'].str.replace('+','').astype(np.int64)
<ipython-input-53-623d5e3ddf8f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [54]:
df_valid_numbers

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,breadcrumb,datepublished,articlebody,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
0,1,Al Bandar Arjaan - Creek,"{'addresscountry': 'United Arab Emirates', 'st...",NaN,AED 224 - AED 2622,{'ratingvalue': '4'},NaN,NaN,25.259316,55.319038,https://mobile.rotana.com/arjaanhotelapartment...,NaN,Baniyas Street,NaN,Dubai,AE,+971 4 704 2299,"{'latitude': '2.5259316E1', 'longitude': '5.53...","{'reviewcount': '1225', 'ratingvalue': '4.68'}",NaN,+971 4 704 2233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97147042299,Country Code: 971 National Number: 47042299,+97147042299,97147042299,0,[0]
1,4,Hala Arjaan by Rotana,"{'addresscountry': 'United Arab Emirates', 'ad...",NaN,AED 168 - AED 395,{'ratingvalue': '4'},NaN,NaN,24.501032,54.377946,https://www.rotana.com/arjaanhotelapartments/u...,NaN,"Hamdan Street, Al Zahiyah Area, PO Box 53066,",NaN,Abu Dhabi,AE,+971 2 418 0000,"{'longitude': '5.437794551253319E1', 'latitude...","{'reviewcount': '1873', 'ratingvalue': '4.48'}",NaN,+971 2 418 0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97124180000,Country Code: 971 National Number: 24180000,+97124180000,97124180000,1,[1]
2,6,ART Rotana,"{'addresscountry': 'Bahrain', 'streetaddress':...",NaN,BHD 28 - BHD 1053,{'ratingvalue': '5'},NaN,NaN,26.289799,50.659563,https://www.rotana.com/rotanahotelandresorts/b...,NaN,"Al Fanar Avenue Road 5722, Building 2780",NaN,Manama,BH,+973 16000111,"{'latitude': '2.6289799107E1', 'longitude': '5...","{'ratingvalue': '4.54', 'reviewcount': '6520'}",NaN,+973 16000112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97316000111,Country Code: 973 National Number: 16000111,+97316000111,97316000111,2,"[2, 32]"
3,8,Rose Rayhaan by Rotana,"{'addresscountry': 'United Arab Emirates', 'ad...",NaN,AED 125 - AED 1906,{'ratingvalue': '4'},NaN,NaN,25.211458,55.277005,https://www.rotana.com/rayhaanhotelandresorts/...,NaN,Sheikh Zayed Road,NaN,Dubai,AE,+971 4 323 0111,"{'longitude': '5.527700453996658E1', 'latitude...","{'reviewcou

In [58]:
df_valid_numbers[df_valid_numbers['indexValue']==49378]

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,breadcrumb,datepublished,articlebody,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
49378,9968,None,"{'postalcode': '32608', 'addresslocality': 'Ga...",NaN,NaN,NaN,NaN,NaN,29.622216,-82.386746,https://www.ihg.com/destinations/de/de/united-...,32608,3370 S.W. 42nd Street,FLORIDA,Gainesville,US,1-352-3781300,"{'longitude': '-8.238674639999999E1', 'latitud...","{'ratingvalue': '4.7', 'bestrating': '5.0', 'r...",None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'name': 'InterContinental Hotels Group'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13523781300,Country Code: 1 National Number: 3523781300,+13523781300,13523781300,49378,"[49071, 49378, 48996]"


In [56]:
len(df_valid_numbers)

49383

In [60]:
# filter out the values which only have one value
data = df_valid_numbers[df_valid_numbers['MatchingNumbers'].apply(lambda x: len(x) > 1)]

In [120]:
len(data)

46646

In [121]:
data.head()

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
0,0,West End Tavern,"{'streetaddress': '926 Pearl St.', 'addressreg...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Boulder,926 Pearl St.,80302,(303) 444-3535,"{'longitude': '-105.283247', 'latitude': '40.0...",None,{'closes': 'false'},US,-105.283247,40.016997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3034443535,Country Code: 1 National Number: 3034443535,+13034443535,13034443535,0,"[0, 50354]"
2,2,Revel,"{'addressregion': 'WA', 'postalcode': '98103',...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,WA,Seattle,403 N 36th Street,98103,(206) 547-2040,"{'longitude': '-122.3562456', 'latitude': '47....",Facebook: Revel Twitter: @revelseattle,{'closes': 'false'},US,-122.356246,47.651964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2065472040,Country Code: 1 National Number: 2065472040,+12065472040,12065472040,2,"[2, 49813]"
5,10,Sweet Action,"{'streetaddress': '52 Broadway', 'postalcode':...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,CO,Denver,52 Broadway,80203,303-282-4645,"{'longitude': '-104.989318', 'latitude': '39.7...",Sweet Action is playing with the boundaries of...,{'closes': 'false'},US,-104.989318,39.717578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3032824645,Country Code: 1 National Number: 3032824645,+13032824645,13032824645,5,"[28299, 5]"
8,15,Uncle Gussy's,"{'streetaddress': 'Park Ave & E 51st St.', 'ad...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,NY,New York,Park Ave & E 51st St.,10022,(929) 600-1111,"{'longitude': '-73.973424', 'latitude': '40.75...",Traditional Greek Food,{'closes': 'false'},US,-73.973424,40.757760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Additional Filtering by Geo Location

In [61]:
def calcDifference(pointOne, pointTwo):
    return geopy.distance.great_circle(pointOne, pointTwo).km

def calcDifferenceFromRow (row):
    tmp = data
    indexValue = row['indexValue']
    indexPosition = (row[lat], row[lon])
    diffList = []
    for value in row['MatchingGeoPoints']:
        if not value in tmp.index:
            continue
        currRow = data.loc[data['indexValue'] == value]
        currIndex = currRow['indexValue'].values[0]
        if currIndex == indexValue:
            diffList.append(-1)
        else:            
            currPosition = (currRow[lat].values[0], currRow[lon].values[0])
            diffList.append(calcDifference(indexPosition, currPosition))
    return diffList


In [62]:
def createKDTree(tupleArray):
    tree = KDTree(tupleArray)
    return tree

# Return all values that are in a specific proximity
def queryTree(tree, point, radius = 0.001):    
    point = [float(i) for i in point]
    idx = tree.query_ball_point(point, r=radius)
    return idx
    #idx = tree.query(point, k=neighbours)
    #return idx[1]

# convert to tuples and from string to float
lonArr = data[lon].to_numpy()
latArr = data[lat].to_numpy()
tupleArray = np.array((lonArr, latArr)).T.astype('float32')

data.reset_index(drop=True, inplace=True)
data['indexValue'] = data.index

# # create new column with all matching points
# tree = createKDTree(tupleArray)
# idx = queryTree(tree, tupleArray[0])

# # Search for the closest neighbour in all of the points
# data['MatchingGeoPoints'] = data.apply(lambda row: queryTree(tree,[row[lon], row[lat]]), axis=1) 

# # Keep those that have one or more matches withing the radius
# data = data[data['MatchingGeoPoints'].apply(lambda x: len(x) > 1)]

<ipython-input-62-42033e8eff04>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['indexValue'] = data.index


In [65]:
data.loc[data['indexValue'] == 5]

,row_id,name,address,description,pricerange,starrating,url,photo,latitude,longitude,page_url,postalcode,streetaddress,addressregion,addresslocality,addresscountry,telephone,geo,aggregaterating,email,faxnumber,alternatename,amenityfeature,availablelanguage,telephone_address,image,ratingvalue,paymentaccepted,brand,review,award,location,reviewbody,author,reviews,photos,containsplace,currenciesaccepted,checkintime,checkouttime,makesoffer,audience,openinghours,interactionstatistic,hasmap,petsallowed,numberofrooms,openinghoursspecification,memberof,parentorganization,breadcrumb,datepublished,articlebody,headline,branchof,map,contactpoint,enddate,startdate,subjectof,branchcode,members,logo,legalname,foundingdate,maximumattendeecapacity,smokingallowed,title,zipcode,offers,hotel-infos-details,hotel-infos-adress,containedinplace,keywords,bestrating,pricecurrency,offeredby,ratingcount,worstrating,special_rate_description,mainentityofpage,founder,sameas,member,containedin,additionalproperty,hotelbrand,hotelcode,reviewcount,releasedate,streetlocality,streetregion,postofficeboxnumber,areaserved,maps,naics,isicv4,itemlistelement,occupancy,acceptsreservations,itemreviewed,dateupdated,employee,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers
5,38,The Boulevard Arjaan by Rotana,"{'addresscountry': 'Jordan', 'addresslocality'...",NaN,JOD 69 - JOD 1430,{'ratingvalue': '5'},NaN,NaN,31.964408,35.906534,https://sw.rotana.com/arjaanhotelapartments/jo...,NaN,"The Boulevard, Rafiq Al Hariri Avenue, Abdali ...",NaN,Amman,JO,+962 6 520 4444,"{'longitude': '3.5906534E1', 'latitude': '3.19...","{'reviewcount': '3264', 'ratingvalue': '4.42'}",NaN,+962 6 520 4555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96265204444,Country Code: 962 National Number: 65204444,+96265204444,96265204444,5,"[12, 7]"


In [124]:
# Calculate the difference in km between those
data['Difference'] = data.apply(lambda row: calcDifferenceFromRow(row), axis=1) 

In [132]:
data.iloc[3:4]

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,MatchingGeoPoints,Difference
32,248,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",NaN,NaN,NaN,NaN,NaN,https://www.cookingchanneltv.com/restaurant-gu...,TX,Austin,4905 Airport Blvd.,78751,(512) 451-3326,"{'longitude': '-97.715239', 'latitude': '30.30...",None,{'closes': 'false'},US,-97.715239,30.30964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5124513326,Country Code: 1 National Number: 5124513326,+15124513326,15124513326,32,"[140, 47716]","[21907, 32]","[0.004145070470485397, -1]"


In [126]:
len(data)

42236

In [146]:
data.loc[data['indexValue'] == 32][['name', 'address', 'page_url', 'E.164 format', lat, lon]]

,name,address,page_url,E.164 format,latitude,longitude
32,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",https://www.cookingchanneltv.com/restaurant-gu...,+15124513326,30.30964,-97.715239


In [138]:
data.loc[data['indexValue'] == 21907]

,row_id,name,address,aggregaterating,opens,closes,validthrough,validfrom,page_url,addressregion,addresslocality,streetaddress,postalcode,telephone,geo,description,openinghoursspecification,addresscountry,longitude,latitude,servescuisine,pricerange,url,hasmenu,acceptsreservations,email,image,faxnumber,openinghours,location,areaserved,paymentaccepted,startdate,enddate,amenityfeature,menu,review,potentialaction,currenciesaccepted,legalname,telephone_address,contactpoint,photo,parentorganization,position,logo,hasmap,containedinplace,website,interactioncount,ratingvalue,ratingcount,rating,category,identifier,photos,sameas,alternatename,ingredients,naics,makesoffer,mainentityofpage,memberof,headline,award,offers,datemodified,event,servicearea,hasoffercatalog,brand,haswifi,foundingdate,employee,numberofemployees,postofficeboxnumber,acceptedpaymentmethod,branchcode,author,additionalproperty,keywords,branchof,containedin,datepublished,dateupdated,address2,price,starrating,publicaccess,text,reviewbody,smokingallowed,publisher,contacttype,employees,phone,bestrating,map,reviews,reviewcount,worstrating,restaurant,region,restaurant-type,occasions,features,interactionstatistic,productid,telephone_,phone_object,E.164 format,telephoneNorm,indexValue,MatchingNumbers,MatchingGeoPoints,Difference
21907,3001,Tyson's Tacos,"{'addresslocality': 'Austin', 'postalcode': '7...",NaN,NaN,NaN,NaN,NaN,https://theinfatuation.com/austin/reviews/tyso...,TX,Austin,4905 Airport Blvd,78751,(512) 451-3326,"{'latitude': '3.0309656E1', 'longitude': '-9.7...",NaN,NaN,US,-97.7152,30.309656,"[Tacos, Tex-Mex]",$,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'value': 'North Loop', 'name': 'Neighborhood'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5124513326,Country Code: 1 National Number: 5124513326,+15124513326,15124513326,21907,"[140, 47716]","[21907, 32]","[-1, 0.00414507047025422]"


In [129]:
pwd

'/work-ceph/bizer-tp2021/data_integration_using_deep_learning/notebooks'

In [130]:
#data.to_json("matchingRestaurant", compression='gzip', orient='records', lines=True)